In [19]:
import pandas as pd 
from utils import seed_it_all

In [20]:
DATA_DIR = "./data"
BATCH = 1
EVENT = 24

In [21]:
seed_it_all(10)

In [22]:
train_meta_df = pd.read_parquet(f'{DATA_DIR}/train_meta.parquet', engine='fastparquet')

In [23]:
batch_df = pd.read_parquet(f'{DATA_DIR}/train/batch_{BATCH}.parquet', engine='fastparquet').reset_index()

In [24]:
sensor_geometry = pd.read_csv(f'{DATA_DIR}/sensor_geometry.csv', index_col=0)

In [25]:
event_df = batch_df[batch_df['event_id'] == EVENT]
event_df = pd.merge(
    left = event_df,
    right = sensor_geometry,
    how='inner',
    on='sensor_id'
)

In [26]:
import numpy as np

x, y, z = event_df['x'], event_df['y'], event_df['z']
coords = np.array((x,y,z)).T
coords

array([[ 303.41,  335.64,  206.58],
       [-145.45,  374.24,  212.73],
       [ 505.27,  257.88, -174.6 ],
       [ 505.27,  257.88, -174.6 ],
       [  -9.68,  -79.5 ,  181.  ],
       [ 576.37,  170.92,  357.88],
       [-290.66, -307.38,  163.61],
       [-526.63,  -15.6 , -178.17],
       [ 500.43,  -58.45,  450.79],
       [-313.6 ,  237.44,  348.01],
       [-313.6 ,  237.44,  348.01],
       [  -9.68,  -79.5 , -205.47],
       [  -9.68,  -79.5 , -205.47],
       [  -9.68,  -79.5 , -205.47],
       [ 576.37,  170.92, -135.72],
       [  -9.68,  -79.5 , -219.49],
       [ 505.27,  257.88,  233.9 ],
       [ 505.27,  257.88,  233.9 ],
       [-234.95,  140.44, -197.79],
       [ 576.37,  170.92,  170.65],
       [-245.65, -190.49,  212.13],
       [-324.39,  -93.43, -350.9 ],
       [-324.39,  -93.43, -350.9 ],
       [ 257.31,  211.66,   44.86],
       [ 124.97, -131.25,  102.18],
       [-324.39,  -93.43,  159.72],
       [ 443.44, -194.35, -181.18],
       [  35.54, -364.83,  1

In [27]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
pca.fit(coords) 
direction_vector = pca.components_
direction_vector

array([[0.88924748, 0.16626122, 0.42614097]])

In [28]:
origin = np.mean(coords, axis=0)
euclidean_distance = np.linalg.norm(coords - origin, axis=1)
extent = np.max(euclidean_distance)
el_1 = origin - direction_vector * extent
el_2 = origin + direction_vector * extent
line = np.vstack((el_1, el_2))
line

array([[-617.22038344, -146.94429322, -275.08081168],
       [ 641.04431786,   88.31150634,  327.89884446]])

In [29]:
import math

def cartesian_to_sphere(x, y, z):
    # https://en.wikipedia.org/wiki/Spherical_coordinate_system
    x2y2 = x**2 + y**2
    r = math.sqrt(x2y2 + z**2)
    azimuth = math.acos(x / math.sqrt(x2y2)) * np.sign(y)
    zenith = math.acos(z / r)
    return azimuth, zenith

In [30]:
def sphere_to_cartesian(azimuth, zenith):
    # see: https://stackoverflow.com/a/10868220/4521646
    x = math.sin(zenith) * math.cos(azimuth)
    y = math.sin(zenith) * math.sin(azimuth)
    z = math.cos(zenith)
    return x, y, z

In [31]:
def adjust_sphere(azimuth, zenith):
    if zenith < 0:
        zenith += math.pi
        azimuth += math.pi
    if azimuth < 0:
        azimuth += math.pi * 2
    azimuth = azimuth % (2 * math.pi)
    return azimuth, zenith

In [32]:

train_metadata=train_meta_df[(train_meta_df['event_id'] == EVENT) & (train_meta_df['batch_id'] == BATCH)]
azimuth = train_metadata['azimuth']
zenith = train_metadata['zenith']
x,y,z = sphere_to_cartesian(azimuth, zenith)
x,y,z

(0.2711606932818947, -0.8260880094077094, -0.4940146547742188)

In [33]:
azimuth_, zenith_ = adjust_sphere(*cartesian_to_sphere(x,y,z))
azimuth_, zenith_

(5.029554633390068, 2.0874975005610428)

In [34]:
train_metadata['azimuth'], train_metadata['zenith']

(0    5.029555
 Name: azimuth, dtype: float64,
 0    2.087498
 Name: zenith, dtype: float64)

In [35]:

azimuth = math.atan2(y, x)
zenith = math.acos(z / math.sqrt(x**2 + y**2 + z**2))
azimuth, zenith

(-1.2536306737895189, 2.0874975005610428)

In [36]:
def get_line(origin, vector):
    below_origin = origin - direction_vector * extent
    above_origin = origin + direction_vector * extent
    line = np.vstack((below_origin, above_origin))
    return line

In [37]:
truth_trace = get_line([0,0,0], [x,y,z])
truth_trace

array([[-629.13235065, -117.62789978, -301.48982807],
       [ 629.13235065,  117.62789978,  301.48982807]])

In [38]:
truth_trace[:,0], truth_trace[:,1], truth_trace[:,2]

(array([-629.13235065,  629.13235065]),
 array([-117.62789978,  117.62789978]),
 array([-301.48982807,  301.48982807]))

## Plot the result

In [39]:
import plotly.graph_objects as go

fig3 = go.Figure(data = [
        go.Scatter3d(
            x=event_df['x'].to_numpy(), y=event_df['y'].to_numpy(), z=event_df['z'].to_numpy(),
            mode='markers',
            marker=dict(size=5, color=event_df['time'].to_numpy(), opacity=1),
            name="Detected"
        ),
        go.Scatter3d(
            x=line[:,0], y=line[:,1], z=line[:,2],
            marker=dict(
                size=4,
                color='red',
            ),
            line=dict(
                color='red',
                width=3
            ),
            name="Predicted"
        ),
        go.Scatter3d(
            x=truth_trace[:,0], y=truth_trace[:,1], z=truth_trace[:,2],
            marker=dict(
                size=4,
                color='green',
            ),
            line=dict(
                color='green',
                width=3
            ),
            name="Truth"
        )
    ],
    
    )
# Add a legend
fig3.update_layout(showlegend=True, legend=dict(x=0, y=1))

fig3.show()

In [40]:
from utils import angular_dist_score

In [41]:
train_metadata

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498


In [42]:
x,y,z

(0.2711606932818947, -0.8260880094077094, -0.4940146547742188)

In [43]:
azimuth = math.atan2(y, x)
zenith = math.acos(z)
azimuth, zenith

(-1.2536306737895189, 2.0874975005610428)